In [1]:
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
import random
from torch.distributions import MultivariateNormal
from torch.optim import Adam

## Network

In [2]:
class FeedForwardNN(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(FeedForwardNN, self).__init__()
        self.layer1 = nn.Linear(in_dim, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, out_dim)

    def forward(self, obs):
        # Convert observation to tensor if it's a numpy array
        if isinstance(obs, np.ndarray):
            obs = torch.tensor(obs, dtype=torch.float)
        
        activation1 = F.relu(self.layer1(obs))
        activation2 = F.relu(self.layer2(activation1))
        output = self.layer3(activation2)
        return output

## PPO

In [3]:
class PPO:
    def __init__(self,obs_dim,act_dim,env):
        self._init_hyperparameters()

        self.env=env
        self.obs_dim=obs_dim
        self.act_dim=act_dim

        self.actor=FeedForwardNN(self.obs_dim,self.act_dim)
        self.critic=FeedForwardNN(self.obs_dim,1)

        self.actor_optim=Adam(self.actor.parameters(),lr=self.lr)
        self.critic_optim=Adam(self.critic.parameters(),lr=self.lr)

        self.cov_var=torch.full(size=(self.act_dim,),fill_value=0.5)
        self.cov_mat=torch.diag(self.cov_var)

    
    def learn(self,total_timesteps):
        t_so_far=0
        while t_so_far<total_timesteps:
            batch_obs,batch_acts,batch_log_probs,batch_rtgs,batch_lens=self.rollout()

            t_so_far+=np.sum(batch_lens)

            V,_=self.evaluate(batch_obs,batch_acts)

            A_k=batch_rtgs-V.detach()

            A_k=(A_k-A_k.mean())/(A_k.std()+1e-10)

            for _ in range(self.n_updates_per_iteration):
                V,curr_log_probs=self.evaluate(batch_obs,batch_acts)
                
                ratios=torch.exp(curr_log_probs-batch_log_probs)

                surr1=ratios*A_k
                surr2=torch.clamp(ratios,1-self.clip,1+self.clip)*A_k

                actor_loss=(-torch.min(surr1,surr2)).mean()
                critic_loss=nn.MSELoss()(V,batch_rtgs)

                self.actor_optim.zero_grad()
                actor_loss.backward(retain_graph=True)
                self.actor_optim.step()

                self.critic_optim.zero_grad()
                critic_loss.backward()
                self.critic_optim.step()

    def evaluate(self,batch_obs,batch_acts):
        V=self.critic(batch_obs).squeeze()

        mean=self.actor(batch_obs)
        dist=MultivariateNormal(mean,self.cov_mat)
        log_probs=dist.log_prob(batch_acts)

        return V,log_probs
            
    def _init_hyperparameters(self):
        self.timesteps_per_batch=24000
        self.max_timesteps_per_episode=1600
        self.gamma=0.995
        self.n_updates_per_iteration=5
        self.clip=0.002
        self.lr=0.0005

        self.set=None

    def rollout(self):
        # Batch data
        batch_obs = []             # batch observations
        batch_acts = []            # batch actions
        batch_log_probs = []       # log probs of each action
        batch_rews = []            # batch rewards
        batch_rtgs = []            # batch rewards-to-go
        batch_lens = []            # episodic lengths in batch

        t=0
        while t<self.timesteps_per_batch:
            ep_rews=[]
            obs=self.env.reset()
            done=False

            G=0
            for ep_t in range(self.max_timesteps_per_episode):
                t+=1
                batch_obs.append(obs)
                action,log_prob=self.get_action(obs)

                ngz_p=[i-min(action) for i in action]
                #print("emmmm:{}".format(ngz_p))
                sampled_action=random.choices([0,1,2,3,4],weights=ngz_p,k=1)
                #print("     why:{}".format(sampled_action))

                reward,obs,done,_=self.env.transition(sampled_action[0])
                G+=reward

                ep_rews.append(reward)
                batch_acts.append(action)
                batch_log_probs.append(log_prob)

                if done:
                    print("done,G=%.2f" % G)
                    break
            
            batch_lens.append(ep_t+1)
            batch_rews.append(ep_rews)
        
        batch_obs=torch.tensor(batch_obs,dtype=torch.float32)
        batch_acts=torch.tensor(batch_acts,dtype=torch.float32)
        batch_log_probs=torch.tensor(batch_log_probs,dtype=torch.float32)

        batch_rtgs=self.compute_rtgs(batch_rews)

        return batch_obs,batch_acts,batch_log_probs,batch_rtgs,batch_lens
    
    def get_action(self,obs):
        mean=self.actor(torch.tensor(obs,dtype=torch.float32))
        dist=MultivariateNormal(mean,self.cov_mat)
        action=dist.sample()
        log_prob=dist.log_prob(action)
        return action.detach().numpy(),log_prob.detach()
    
    def compute_rtgs(self,batch_rews):
        batch_rtgs=[]

        for ep_rews in reversed(batch_rews):
            discounted_reward=0
            for reward in reversed(ep_rews):
                discounted_reward=reward+discounted_reward*self.gamma
                batch_rtgs.insert(0,discounted_reward)
        
        batch_rtgs=torch.tensor(batch_rtgs,dtype=torch.float32)
        return batch_rtgs

## Env

In [4]:
class Agent:
    def __init__(self,n_actions,n_observations):
        pass
    
    def act(self, state):
        # Simple-minded agent that always select action 1
        return 1

In [5]:
def stringStateToIntState(state):
    tyreDict={}
    tyreDict["Ultrasoft"]=0
    tyreDict["Soft"]=1
    tyreDict["Intermediate"]=2
    tyreDict["Fullwet"]=3

    weatherDict={}
    weatherDict["Dry"]=0
    weatherDict["20% Wet"]=1
    weatherDict["40% Wet"]=2
    weatherDict["60% Wet"]=3
    weatherDict["80% Wet"]=4
    weatherDict["100% Wet"]=5

    return([tyreDict[state[0]],state[1],weatherDict[state[2]],state[3],state[4]])



class Car:
    def __init__(self, tyre="Intermediate"):
        self.default_tyre = tyre
        self.possible_tyres = ["Ultrasoft", "Soft", "Intermediate", "Fullwet"]
        self.pitstop_time = 23
        self.reset()
    
    
    def reset(self):
        self.change_tyre(self.default_tyre)
    
    
    def degrade(self, w, r):
        if self.tyre == "Ultrasoft":
            self.condition *= (1 - 0.0050*w - (2500-r)/90000)
        elif self.tyre == "Soft":
            self.condition *= (1 - 0.0051*w - (2500-r)/93000)
        elif self.tyre == "Intermediate":
            self.condition *= (1 - 0.0052*abs(0.5-w) - (2500-r)/95000)
        elif self.tyre == "Fullwet":
            self.condition *= (1 - 0.0053*(1-w) - (2500-r)/97000)
        
        
    def change_tyre(self, new_tyre):
        assert new_tyre in self.possible_tyres
        self.tyre = new_tyre
        self.condition = 1.00
    
    
    def get_velocity(self):
        if self.tyre == "Ultrasoft":
            vel = 80.7*(0.2 + 0.8*self.condition**1.5)
        elif self.tyre == "Soft":
            vel = 80.1*(0.2 + 0.8*self.condition**1.5)
        elif self.tyre == "Intermediate":
            vel = 79.5*(0.2 + 0.8*self.condition**1.5)
        elif self.tyre == "Fullwet":
            vel = 79.0*(0.2 + 0.8*self.condition**1.5)
        return vel

    
class Track:
    def __init__(self, car=Car()):
        # self.radius and self.cur_weather are defined in self.reset()
        self.total_laps = 162
        self.car = car
        self.possible_weather = ["Dry", "20% Wet", "40% Wet", "60% Wet", "80% Wet", "100% Wet"]
        self.wetness = {
            "Dry": 0.00, "20% Wet": 0.20, "40% Wet": 0.40, "60% Wet": 0.60, "80% Wet": 0.80, "100% Wet": 1.00
        }
        self.p_transition = {
            "Dry": {
                "Dry": 0.987, "20% Wet": 0.013, "40% Wet": 0.000, "60% Wet": 0.000, "80% Wet": 0.000, "100% Wet": 0.000
            },
            "20% Wet": {
                "Dry": 0.012, "20% Wet": 0.975, "40% Wet": 0.013, "60% Wet": 0.000, "80% Wet": 0.000, "100% Wet": 0.000
            },
            "40% Wet": {
                "Dry": 0.000, "20% Wet": 0.012, "40% Wet": 0.975, "60% Wet": 0.013, "80% Wet": 0.000, "100% Wet": 0.000
            },
            "60% Wet": {
                "Dry": 0.000, "20% Wet": 0.000, "40% Wet": 0.012, "60% Wet": 0.975, "80% Wet": 0.013, "100% Wet": 0.000
            },
            "80% Wet": {
                "Dry": 0.000, "20% Wet": 0.000, "40% Wet": 0.000, "60% Wet": 0.012, "80% Wet": 0.975, "100% Wet": 0.013
            },
            "100% Wet": {
                "Dry": 0.000, "20% Wet": 0.000, "40% Wet": 0.000, "60% Wet": 0.000, "80% Wet": 0.012, "100% Wet": 0.988
            }
        }
        self.reset()
    
    
    def reset(self):
        self.radius = 1200#np.random.randint(600,1201)
        self.cur_weather = np.random.choice(self.possible_weather)
        self.is_done = False
        self.pitstop = False
        self.laps_cleared = 0
        self.car.reset()
        return stringStateToIntState(self._get_state())
    
    
    def _get_state(self):
        return [self.car.tyre, self.car.condition, self.cur_weather, self.radius, self.laps_cleared]
        
    
    def transition(self, action=0):
        """
        Args:
            action (int):
                0. Make a pitstop and fit new ‘Ultrasoft’ tyres
                1. Make a pitstop and fit new ‘Soft’ tyres
                2. Make a pitstop and fit new ‘Intermediate’ tyres
                3. Make a pitstop and fit new ‘Fullwet’ tyres
                4. Continue the next lap without changing tyres
        """
        ## Pitstop time will be added on the first eight of the subsequent lap
        time_taken = 0
        if self.laps_cleared == int(self.laps_cleared):
            if self.pitstop:
                self.car.change_tyre(self.committed_tyre)
                time_taken += self.car.pitstop_time
                self.pitstop = False
        
        ## The environment is coded such that only an action taken at the start of the three-quarters mark of each lap matters
        if self.laps_cleared - int(self.laps_cleared) == 0.75:
            if action < 4:
                self.pitstop = True
                self.committed_tyre = self.car.possible_tyres[action]
            else:
                self.pitstop = False
        
        self.cur_weather = np.random.choice(
            self.possible_weather, p=list(self.p_transition[self.cur_weather].values())
        )
        # we assume that degration happens only after a car has travelled the one-eighth lap
        velocity = self.car.get_velocity()
        time_taken += (2*np.pi*self.radius/8) / velocity
        reward = 0-time_taken
        #reward=0-action
        self.car.degrade(
            w=self.wetness[self.cur_weather], r=self.radius
        )
        self.laps_cleared += 0.125
        
        if self.laps_cleared == self.total_laps:
            self.is_done = True
        
        next_state = stringStateToIntState(self._get_state())
        return reward, next_state, self.is_done, velocity

In [6]:
new_car = Car()
env = Track(new_car)

agent = Agent(5,5)

In [7]:
state = env.reset()    
done = False
G = 0
while not done:
    action = agent.act(state)
    reward, next_state, done, velocity = env.transition(action)
    # added velocity for sanity check
    state = deepcopy(next_state)
    G += reward

print("G: %d" % G)

G: -19977


In [8]:
agent=PPO(5,5,env)
agent.learn(100000000)

done,G=-19903.26
done,G=-19938.18
done,G=-20056.45
done,G=-19954.44
done,G=-20429.96
done,G=-19938.55
done,G=-19880.90
done,G=-19965.88
done,G=-20038.94
done,G=-20084.08
done,G=-20026.11
done,G=-19963.78
done,G=-20037.14
done,G=-19990.54
done,G=-19907.28
done,G=-20051.19
done,G=-20087.23
done,G=-20091.22
done,G=-20122.48


/tmp/ipykernel_8571/1441262019.py:111: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  batch_acts=torch.tensor(batch_acts,dtype=torch.float32)


done,G=-20103.71
done,G=-20355.33
done,G=-20052.86
done,G=-20167.28
done,G=-19721.47
done,G=-19941.18
done,G=-20095.28
done,G=-20131.86
done,G=-20319.11
done,G=-19889.54
done,G=-20237.18
done,G=-20136.60
done,G=-20967.25
done,G=-20138.50
done,G=-20368.21
done,G=-20103.05
done,G=-20356.54
done,G=-20262.58
done,G=-20404.75
done,G=-20064.06
done,G=-19996.18
done,G=-20300.28
done,G=-20205.56
done,G=-20218.16
done,G=-20308.77
done,G=-19894.91
done,G=-19939.07
done,G=-20492.58
done,G=-19950.94
done,G=-20089.08
done,G=-20157.05
done,G=-19994.19
done,G=-19862.59
done,G=-20001.58
done,G=-20021.12
done,G=-20017.43
done,G=-20054.86
done,G=-20391.88
done,G=-20119.85
done,G=-20198.30
done,G=-20212.62
done,G=-19829.68
done,G=-20517.43
done,G=-19860.08
done,G=-20073.98
done,G=-20011.48
done,G=-20150.22
done,G=-20426.80
done,G=-20541.72
done,G=-20037.54
done,G=-19998.89
done,G=-20032.82
done,G=-19901.10
done,G=-20128.09
done,G=-20052.92
done,G=-19795.91
done,G=-20304.18
done,G=-19980.64
done,G=-20216.